|<h2>Book:</h2>|<h1><a href="https://open.substack.com/pub/mikexcohen/p/llm-breakdown-16-tokenization-words" target="_blank">50 ML projects to understand LLMs</a></h1>|
|-|:-:|
|<h2>Project:</h2>|<h1><b>[28] Effective dimensionality of hidden layers</b></h1>|
|<h2>Author:<h2>|<h1>Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h1>|

<br>

<i>Using the code without reading the book may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import requests

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
### matplotlib adjustments (commented lines are for dark mode)

# svg plots (higher-res)
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

plt.rcParams.update({
    # 'figure.facecolor': '#282a2c',
    # 'figure.edgecolor': '#282a2c',
    # 'axes.facecolor':   '#282a2c',
    # 'axes.edgecolor':   '#DDE2F4',
    # 'axes.labelcolor':  '#DDE2F4',
    # 'xtick.color':      '#DDE2F4',
    # 'ytick.color':      '#DDE2F4',
    # 'text.color':       '#DDE2F4',
    'axes.spines.right': False,
    'axes.spines.top':   False,
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold',
    'savefig.dpi':300
})

# **Concept of effective dimensionality**

In [ ]:
# a bit of data
x = np.random.uniform(low=0,high=5,size=50)
y = np.random.normal(loc=0,scale=.07,size=len(x))

_,axs = plt.subplots(1,2,figsize=(9,3.5))
axs[0].plot(x,x,'ko',markerfacecolor=[.9,.7,.7,.5],markersize=10)
axs[1].plot(x,x+y,'ko',markerfacecolor=[.7,.9,.9,.5],markersize=10)

for a in axs:
  a.axis('square')
  a.plot([0,5],[0,5],color='gray',zorder=-2)

axs[0].set(xlabel='x',ylabel='y',title='A) 1-dimensional data')
axs[1].set(xlabel='x',ylabel='y',title='B) 1 effective dimension')

plt.tight_layout()
plt.savefig('ch5_proj28_part0.png')
plt.show()

# **Part 1: Logits from real and shuffled token sequences**

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# load GPT2 model and tokenizer
model = AutoModelForCausalLM.from_pretrained('gpt2-xl')
tokenizer = AutoTokenizer.from_pretrained('gpt2-xl')

max_seq_len =

model.to(device)
model.eval()

In [ ]:
### use this cell for Part 4
# from transformers import AutoModelForCausalLM, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained('EleutherAI/pythia-2.8b')
# model = AutoModelForCausalLM.from_pretrained('EleutherAI/pythia-2.8b')

# max_seq_len =

# model.to(device)
# model.eval()

In [ ]:
# Through the looking glass (Alice in Wonderland)
text = requests.get('https://www.gutenberg.org/cache/epub/11/pg11.txt').text

# for Part 6
# text = requests.get('https://www.gutenberg.org/').text


allTokens =

# get context-length from middle of the book
start_idx =
end_idx =

tokens = allTokens[]

print(tokenizer.decode(tokens[0]))

In [ ]:
tokensShuffle = tokens[]
print(tokenizer.decode(tokensShuffle[0]))

In [ ]:
# push through the model (~3 mins with gpt2-xl on standard CPU, or <1s on GPU, lol)
with torch.no_grad():
  outputs_real = model(
  outputs_shuf = model(

outputs_real.hidden_states[0].shape

In [ ]:
# calculate log-softmax of logits (note the method instead of function)
log_sm_real = outputs_real.logits.log_softmax
log_sm_shuf = outputs_shuf.logits.log_softmax

# find the indices of the max log-sm values
maxidx = torch.argmax(log_sm_real,

# then find the actual log-sm values from those indices
maxvals = []

# and calculate the histogram
yReal,xReal = np.histogram(


### repeat for shuffled
maxidx =
maxvals =
yShuf,xShuf =


# visualize
plt.figure(figsize=(9,3))
plt.plot(xReal[:-1],yReal,'bs-',markerfacecolor=[.7,.7,.9],markersize=12,linewidth=2,label='Real')
plt.plot(label='Shuffled')

plt.gca().set(xlabel='Max log softmax',ylabel='Density',title='Maximum token probabilities')
plt.legend()

plt.tight_layout()
plt.savefig('ch5_proj28_part1.png')
plt.show()

# **Part 2: Effective dimensionality in one layer (numpy)**

In [ ]:
layeri = 10

# extract all the activations from this layer
acts = outputs_real.

# mean-center the activations
acts -=

# get singular values
s = np.linalg.svd

# percent explained (and cumulative)
pctExplained =
cumVarExplained =


_,axs = plt.subplots(1,2,figsize=(10,3))

axs[0].plot(pctExplained,'ks',markersize=10,markerfacecolor=[.7,.9,.7,.7])
axs[0].set(xlim=[-2,60],xlabel='Component number',ylabel='Percent variance explained',
           title='A) Singular values spectrum')

axs[1].plot(cumVarExplained,'ks',markersize=10,markerfacecolor=[.7,.7,.9,.7])


plt.tight_layout()
plt.savefig('ch5_proj28_part2.png')
plt.show()

In [ ]:
threshold = 95

# count the components until 95% variance is explained
effectiveCompCount =
print(f'{} (out of {}) components explains {} variability')

# **Part 3: Laminar profile of effective dimensionality (PyTorch)**

In [ ]:
numHidden = len(outputs_real.hidden_states)
numHidden

In [ ]:
# redefine threshold
threshold = 95

# initialize
effectiveCompCount = torch.zeros((,),dtype=)
hs_ranks = torch.zeros((,),dtype=)


# loop over layers
for layeri in tqdm(range(numHidden)):

  # extract all the activations from this layer (assuming no batches!)
  acts = outputs_real.

  # mean-center the activations and calcuate rank
  acts -=
  hs_ranks[layeri,0] = torch.linalg.

  # get singular values
  s = torch.linalg.

  # percent explained (cumulative)
  pctExplained =
  cumVarExplained =

  # count the components until 95% variance is explained
  compcount =
  effectiveCompCount[layeri,0] =



  ### repeat for shuffled tokens
  acts =
  hs_ranks[layeri,1] =
  s = # get singular values
  pctExplained =
  cumVarExplained =
  effectiveCompCount[layeri,1] =


In [ ]:
# for reference with the scatter plot
print(f'There are {} out of {} unique tokens.')

In [ ]:
_,ax = plt.subplots(1,figsize=(10,3))

## plot the "effective subspace dimensionality" of each layer
ax.plot(,'ks',markerfacecolor=[.9,.7,.7,.5],markersize=12,label='Shuffled tokens')
ax.plot(,label='Real tokens')
ax.legend()
ax.set(xlabel='Hidden layer',ylabel='Number of dimensions',title='Maximum possible dimensionality',xlim=[-1,numHidden])

plt.tight_layout()
plt.savefig('ch5_proj28_part3a.png')
plt.show()

In [ ]:
hs_ranks

In [ ]:
_,ax = plt.subplots(1,figsize=(12,4))

## plot the "effective subspace dimensionality" of each layer
ax.plot(label='Shuffled tokens')
ax.plot(label='Real tokens')

ax2 = ax.twinx()
a = 100*effectiveCompCount[:,1] /
b =
ax2.plot(a,linewidth=2,color=[.9,.7,.7])
ax2.plot(b,linewidth=2,color=[.7,.9,.7])
ax2.set(ylabel='% max dimensionality')

plt.tight_layout()
plt.savefig('ch5_proj28_part3b.png')
plt.show()

# **Part 4: Repeat with Pythia-2.8b**

# **Part 5: Repeat with html code**